# Inventory Data Transformation
Note: Some columns may be automatically omitted from the notebook for display purposes, but still exist in the table itself.
## Import data
Data from CSVs is loaded into working dataframes.

In [86]:
import pandas as pd
import glob
import os

data = pd.read_csv('data/fake/001_edited.csv')
cost_data = pd.read_csv('data/fake/Costs_edited.csv', low_memory=False)
data.head()

,PART NUMBER,PART DESCRIPTION,Unnamed: 2,closure
0,XXX-001,SHEET - 40/60 8.5 X 11 X .15,plastic,NaN
1,XXX-002,SHEET - 40/60 CLEAR 8 1/2 X 11 X .095 DO NOT USE,plastic,NaN
2,XXX-003,SHEET - 40/60 CLEAR 8.5 X 11 7/8 X .09 (G23453),plastic,NaN
3,XXX-004,SHEET - 25 X 36.25 X .177 CLEAR DURAYL 40/60,plastic,NaN
4,XXX-005,SHEET - 40/60 CLEAR 32 X 35 1/4 X .077 (OBSOLETE),plastic,NaN


## Clean data
Any rows containing product numbers without an assigned product are removed. For example, a PartNumber without a corresponding description is removed, since there is no associated part. Additionally, rows containing any part numbers marked as 'RESERVED' or 'DO NOT USE' were removed.

In [87]:
print('Number of rows in data before clean:')
print(len(data))

# Remove junk rows
for index, row in data.iterrows():
    no_part_description = type(row['PART DESCRIPTION']) == float
    no_part_number = type(row['PART NUMBER']) == float and type(row['Unnamed: 2']) == float
    if no_part_description:
        data = data.drop([index])
        continue
    empty_description = len(row['PART DESCRIPTION']) < 2
    invalid_description = ('RESERVED' in row['PART DESCRIPTION'] or 'DO NOT USE' in row['PART DESCRIPTION']) and len(
        row['PART DESCRIPTION']) <= 11 or 'RESERVED BOR HAULER 4 X 4 BALL CAGE PARTS' in row['PART DESCRIPTION']
    if no_part_number or empty_description or invalid_description:
        data = data.drop([index])

print('and after clean:')
print(len(data))

Number of rows in data before clean:
2538
and after clean:
2460


## Create output table
The output table containing the 49 desired columns is created. No data has been moved into the table at this point.

In [88]:
data_new = pd.DataFrame(
    columns=['PartNumber', 'PartDescription', 'PartDetails', 'UOM', 'UPC', 'PartTypeID', 'Active', 'StdCost',
             'Tracks-Lot Number', 'Tracks-Revision Level', 'Tracks-Expiration Date', 'Tracks-Serial Number',
             'AssetAccount', 'COGSAccount', 'AdjustmentAccount', 'ScrapAccount', 'VarianceAccount', 'ABCCode', 'Weight',
             'WeightUOM', 'Width', 'Height', 'Len', 'SizeUOM', 'PartURL', 'PartRevision', 'CF-Custom', 'ProductNumber',
             'ProductDescription', 'ProductDetails', 'Price', 'ProductSKU', 'ProductUPC', 'ProductActive',
             'ProductTaxable', 'ProductSOItemTypeID', 'IncomeAccount', 'ProductWeight', 'ProductWeightUOM',
             'ProductWidth', 'ProductHeight', 'ProductLength', 'ProductSizeUOM', 'Vendor', 'DefaultVendor',
             'VendorPartNumber', 'Cost', 'VendorUOM', 'CFP-Custom'])
data_new.head()

,PartNumber,PartDescription,PartDetails,UOM,UPC,PartTypeID,Active,StdCost,Tracks-Lot Number,Tracks-Revision Level,...,ProductWidth,ProductHeight,ProductLength,ProductSizeUOM,Vendor,DefaultVendor,VendorPartNumber,Cost,VendorUOM,CFP-Custom


## Migrate data
### The easy stuff is migrated first:
* Product numbers and descriptions
* Data consistent across all rows:
    * UOM
    * Tracks-Expiration Date

In [89]:
# Static fields & easily migrated
data_new['PartNumber'] = data['PART NUMBER'].copy()
data_new['ProductNumber'] = data['PART NUMBER'].copy()
data_new['UOM'] = 'ea'
data_new['Tracks-Expiration Date'] = False
data_new['PartDescription'] = data['PART DESCRIPTION'].copy()
data_new['ProductDescription'] = data['PART DESCRIPTION'].copy()
data_new['PartDetails'] = data['Unnamed: 2'].copy()
data_new_pr = data_new[['PartNumber', 'ProductNumber', 'UOM', 'Tracks-Expiration Date', 'PartDescription',
                        'PartDetails']]
data_new_pr.head()

,PartNumber,ProductNumber,UOM,Tracks-Expiration Date,PartDescription,PartDetails
0,XXX-001,XXX-001,ea,False,SHEET - 40/60 8.5 X 11 X .15,plastic
1,XXX-002,XXX-002,ea,False,SHEET - 40/60 CLEAR 8 1/2 X 11 X .095 DO NOT USE,plastic
2,XXX-003,XXX-003,ea,False,SHEET - 40/60 CLEAR 8.5 X 11 7/8 X .09 (G23453),plastic
3,XXX-004,XXX-004,ea,False,SHEET - 25 X 36.25 X .177 CLEAR DURAYL 40/60,plastic
4,XXX-005,XXX-005,ea,False,SHEET - 40/60 CLEAR 32 X 35 1/4 X .077 (OBSOLETE),plastic


### Then data requiring more detailed parsing (processing of text):
#### Active status
Whether a product is marked as active (represented by a boolean value of either 'TRUE' or 'FALSE' in the table) is determined by if the product description contains any of the following keywords (lower or uppercase): do not use, obsolete, no longer use.

In [90]:
words_active = {'obsolete', 'do not use', 'no longer use'}

# Set products' active status, based on if the part description contains 'obsolete' or 'do not use'.
for index, row in data.iterrows():
    if any(map(row['PART DESCRIPTION'].lower().__contains__, words_active)) or any(
            map(str(row['Unnamed: 2']).lower().__contains__, words_active)):
        data_new.loc[index, 'Active'] = 'FALSE'
        data_new.loc[index, 'ProductActive'] = 'FALSE'
    else:
        data_new.loc[index, 'Active'] = 'TRUE'
        data_new.loc[index, 'ProductActive'] = 'TRUE'

data_new_active_pr = data_new[['PartNumber', 'PartDescription', 'Active', 'ProductActive']]
data_new_active_pr.head()

,PartNumber,PartDescription,Active,ProductActive
0,XXX-001,SHEET - 40/60 8.5 X 11 X .15,TRUE,TRUE
1,XXX-002,SHEET - 40/60 CLEAR 8 1/2 X 11 X .095 DO NOT USE,FALSE,FALSE
2,XXX-003,SHEET - 40/60 CLEAR 8.5 X 11 7/8 X .09 (G23453),TRUE,TRUE
3,XXX-004,SHEET - 25 X 36.25 X .177 CLEAR DURAYL 40/60,TRUE,TRUE
4,XXX-005,SHEET - 40/60 CLEAR 32 X 35 1/4 X .077 (OBSOLETE),FALSE,FALSE


#### Cost of product
The StdCost of the product is gathered from the total cost of each product listed in the spreadsheet (the 'TOTAL COST' column) containing product costs. Not every part number was listed in the cost data, which is why some costs are missing (or show up as 'NaN').

In [91]:
# Cost data
c = {'cost': cost_data[' TOTAL COST '], 'part': cost_data['PART NUMBER']}
costs = pd.DataFrame(data=c)
data_incl_cost = pd.merge(data_new, costs, how="left", left_on='PartNumber', right_on='part')
data_incl_cost['StdCost'] = data_incl_cost['cost'].copy()
data_incl_cost = data_incl_cost.drop(columns=['cost', 'part'])
data_incl_cost_pr = data_incl_cost[['PartNumber', 'StdCost']]
data_incl_cost_pr.head()

,PartNumber,StdCost
0,XXX-001,NaN
1,XXX-002,NaN
2,XXX-003,NaN
3,XXX-004,$20.21
4,XXX-005,NaN


#### Income, COGS, and asset accounts
The various accounts are migrated to the new spreadsheet using a separate datasheet containing account info for various parts. Accounts are migrated to the new spreadsheet by matching rows based on part number.

Additionally, some rows contain symbols not recognized. Those rows are cleaned prior to migration.

In [92]:
import regex as re

accounts_data = pd.read_csv('data/fake/Items and Services export_e.CSV', low_memory=False)
cols = ['Account', 'COGS Account', 'Asset Account']
accounts_data[cols] = accounts_data[cols].replace({'�':''}, regex=True)
accounts_raw = {'part': accounts_data['Item'], 'account': accounts_data['Account'], 'cogs': accounts_data['COGS Account'],
            'asset': accounts_data['Asset Account']}
accounts = pd.DataFrame(data=accounts_raw)
data_incl_cost = pd.merge(data_incl_cost, accounts, how="left", left_on='PartNumber', right_on='part')
data_incl_cost['AssetAccount'] = data_incl_cost['asset'].copy()
data_incl_cost['COGSAccount'] = data_incl_cost['cogs'].copy()
data_incl_cost['IncomeAccount'] = data_incl_cost['account'].copy()
data_incl_cost = data_incl_cost.drop(columns=['asset', 'cogs', 'account'])
data_incl_cost_acct_pr = data_incl_cost[['PartNumber', 'AssetAccount', 'COGSAccount', 'IncomeAccount']]
data_incl_cost_acct_pr.head()

,PartNumber,AssetAccount,COGSAccount,IncomeAccount
0,XXX-001,NaN,NaN,NaN
1,XXX-002,NaN,NaN,NaN
2,XXX-003,NaN,NaN,NaN
3,XXX-004,INVENTORY ASSET:PLASTIC:3115.0 Windshields / ...,RM:PLASTIC:1005.0 Windshields / Tops COGS,SALES:PLASTIC:5020.0 Windshields / Tops Sales
4,XXX-005,NaN,NaN,NaN


#### PartTypeID
The PartTypeID has two possible values:
* '40': representing office supplies
* '10': representing anything else

Most parts in the inventory are not office supplies, so manually picking out office supply entries would be tedious. To separate any office supply entries, any row that contains office supply keywords, such as paper, staple, container, pencil, etc., are marked as an office supply, provided they are after part 204-084. Anything before that is marked with '10'.

Below is a percentage and count of inventory that is made up by office supplies (as classified by this program), as a check for accuracy.

In [93]:
supl = {'paper', 'staple', 'container', 'pencil', ' pen', 'marker', 'eraser', 'folder', 'notebook', 'stapler',
        'scissors', 'envelope',}
count = 0

for index, row in data_incl_cost.iterrows():
    if index < 7244:
        data_incl_cost.loc[index, 'PartTypeID'] = '10'
    elif any(map(row['ProductDescription'].lower().__contains__, supl)):
        data_incl_cost.loc[index, 'PartTypeID'] = '40'
        count += 1
    else: data_incl_cost.loc[index, 'PartTypeID'] = '10'

print(f'Percentage of inventory made up of office supplies: {count / len(data_incl_cost)}')
print(f'Marked as Office Supplies: {count}\tTotal: {len(data_incl_cost)}')

Percentage of inventory made up of office supplies: 0.0
Marked as Office Supplies: 0	Total: 2468


#### VendorPartNumber
VendorPartNumbers are obtained by parsing anything that exists in a set of parenthesis in each part description. In the case of there being multiple vendor part numbers, each additional vendor part number is appended to the first, separated by a comma.

To determine whether text contained in parentheses (I'll refer to this as "string in question") is a vendor part number, the following conditions were used to exclude any string in question from being considered as a vendor part number:
* length is less than 6
* does not contain any numbers
* contains a measurement, usually has any of the following: '"' (double quotation mark), ' X ' (including space before and after), 'THICK'/'THK', 'LB', or  'COMPRESSION'
* contains other specification, usually has: 'COLOR', 'DIE', or 'POLY'
* contains a note, not a model number: 'INC', 'USE', 'REPLACE', or '2EA'
* contains other words not associated with part numbers in the same string in question: 'CLUB', 'CUSHMAN', or 'DUROMETER'

If the string failed to be excluded using those conditions, it is considered to be a vendor part number and is included in the appropriate cell.

Additionally, text outside of parentheses was parsed. Any slashes were removed from the descriptions, then any text after "PN" was considered to be a part number, and were appended to the associated row under VendorPartNumber.

In [94]:
# Definite exclusion keywords:
words_docs = {'"', ' X ', ' x ', 'COLOR', 'DIE', 'THICK', 'THK', 'POLY', 'INC', 'USE', 'CLUB', 'REPLACE', 'LB',
              'CUSHMAN', 'COMPRESSION', 'DUROMETER', '2EA', 'INCLUDES'}

for index, row in data_incl_cost.iterrows():
    doc = None
    data_incl_cost.loc[index, 'VendorPartNumber'] = ''
    for string in re.findall('(\(.*?\))', row['ProductDescription']):
        doc = re.sub('[(/)]', '', string)
        if (len(doc) < 6) or (not (any(char.isdigit() for char in doc))) or re.search('\dX\d', doc) or (
                any(map(doc.__contains__, words_docs))):
            continue
        if len(data_incl_cost.loc[index, 'VendorPartNumber']) > 1: data_incl_cost.loc[index, 'VendorPartNumber'] += ', '
        data_incl_cost.loc[index, 'VendorPartNumber'] += doc

    for string in re.findall(r'(.*?)\(.*?\)', row['ProductDescription']):
        doc = re.sub('[(/)]', '', string)
        if len(doc.split("PN", 1)) > 1:
            doc = doc.split("PN", 1)[1]
            data_incl_cost.loc[index, 'VendorPartNumber'] += ', '
            data_incl_cost.loc[index, 'VendorPartNumber'] += doc

vendor_pn_pr = data_incl_cost[['PartNumber', 'PartDescription', 'VendorPartNumber']]
vendor_pn_pr.head()

,PartNumber,PartDescription,VendorPartNumber
0,XXX-001,SHEET - 40/60 8.5 X 11 X .15,
1,XXX-002,SHEET - 40/60 CLEAR 8 1/2 X 11 X .095 DO NOT USE,
2,XXX-003,SHEET - 40/60 CLEAR 8.5 X 11 7/8 X .09 (G23453),G23453
3,XXX-004,SHEET - 25 X 36.25 X .177 CLEAR DURAYL 40/60,
4,XXX-005,SHEET - 40/60 CLEAR 32 X 35 1/4 X .077 (OBSOLETE),


### Image File Paths
File paths to part drawings for each part that isn't an office supply are needed in the PartURL column. To do so, '.dwg' is appended to the part number, appended to the directory where drawings are stored.

In [95]:
for index, row in data_incl_cost.iterrows():
    if row['PartTypeID'] == '10': data_incl_cost.loc[
        index, 'PartURL'] = rf"F:/Current Part Drawings/{row['PartNumber']}.dwg"

filepath_pr = data_incl_cost[['PartNumber', 'PartTypeID', 'PartDescription', 'PartURL']]
filepath_pr.head()

,PartNumber,PartTypeID,PartDescription,PartURL
0,XXX-001,10,SHEET - 40/60 8.5 X 11 X .15,F:/Current Part Drawings/XXX-001.dwg
1,XXX-002,10,SHEET - 40/60 CLEAR 8 1/2 X 11 X .095 DO NOT USE,F:/Current Part Drawings/XXX-002.dwg
2,XXX-003,10,SHEET - 40/60 CLEAR 8.5 X 11 7/8 X .09 (G23453),F:/Current Part Drawings/XXX-003.dwg
3,XXX-004,10,SHEET - 25 X 36.25 X .177 CLEAR DURAYL 40/60,F:/Current Part Drawings/XXX-004.dwg
4,XXX-005,10,SHEET - 40/60 CLEAR 32 X 35 1/4 X .077 (OBSOLETE),F:/Current Part Drawings/XXX-005.dwg


### Export to CSV
The exported CSV in its final format is not available to the public for confidentiality.

In [96]:
# Output file name
filename = 'PPVendorPricing_transformed.csv'

data_incl_cost.to_csv(filename, index=False)
print(f'Table exported to {filename}')

Table exported to PPVendorPricing_transformed.csv
